# Lecture 4

## Example: linear shape functions and approximation over one element 
Consider an approximation over one element $x \in (0, 1)$ using two degrees of freedom
$$
    u_h(x) \approx N_1(x)\,a_1 + N_2(x)\,a_2
$$
where the shape functions are
$$
    N_1(x) = 1-x, \qquad N_2(x) = x
$$
Study how the approximation $u_h(x)$ varies when changing the DOFS. 

In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from plotly.io import renderers

# If LaTeX doesn't render in VS Code, try: renderers.default = "iframe_connected"
renderers.default = "notebook_connected"

# ---------------- User settings ----------------
HAT_LINE_WIDTH = 1.5      # thickness for each (unscaled) hat N_i (dashed)
UH_LINE_WIDTH  = 4.0      # thickness for u_h(x) (solid)
PALETTE = ["#d62728", "#2ca02c"]
SHOW_NODE_LABELS = True   # annotate nodes 1..nn under the axis
YRANGE = (-2.1, 2.1)      # adjust if you widen DOF slider range

# ---------------- Mesh (1 element -> 2 nodes) ----------------
nodes = np.array([0.0, 1.0])   # single element [0,1]
nn = len(nodes)                 # 2
x = np.linspace(0.0, 1.0, 1000) # dense grid on the single element

# ---------------- Global P1 (hat) basis ----------------
def hat_basis(i, x, nodes):
    """
    Global P1 hat function at node i (support on [x_{i-1}, x_{i+1}]).
    For 1 element, N1(x)=1-x, N2(x)=x on [0,1].
    """
    N = np.zeros_like(x, dtype=float)
    xi = nodes[i]
    if i > 0:
        m = (x >= nodes[i-1]) & (x <= xi)
        # for i=1: from node 0 to node 1, descending to xi
        N[m] = (x[m] - nodes[i-1]) / (xi - nodes[i-1])
    if i < len(nodes) - 1:
        m = (x >= xi) & (x <= nodes[i+1])
        # for i=0: from xi to node 1, descending from 1 to 0
        N[m] = (nodes[i+1] - x[m]) / (nodes[i+1] - xi)
    return N

# Precompute the two hats
N_global = np.vstack([hat_basis(i, x, nodes) for i in range(nn)])  # shape (2, len(x))

def assemble_u(a):
    """u_h(x) = sum_i a_i * N_i(x) on the dense grid."""
    return a @ N_global

# ---------------- Initial DOFs ----------------
a0 = np.array([0.5, 1.4])   # a1, a2

# ---------------- Figure ----------------
fig = go.FigureWidget(layout=dict(
    title="u_h(x) = N₁a₁ + N₂a₂ = (1-x)a₁ + (x)a₂ ",
    xaxis_title="x",
    yaxis_title="value",
    template="plotly_white",
    height=520
))

# 1) Unscaled hats N_i(x), dashed
hat_trace_indices = []
for i in range(nn):
    tr_idx = len(fig.data)
    fig.add_scatter(
        x=x, y=N_global[i],
        mode="lines",
        name=f"N_{i+1}(x)",  # 1-based label
        line=dict(width=HAT_LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid")
    )
    hat_trace_indices.append(tr_idx)

# 2) u_h(x) = Σ a_i N_i(x), solid
idx_uh = len(fig.data)
fig.add_scatter(
    x=x, y=assemble_u(a0),
    mode="lines",
    name="u_h(x) = a₁N₁ + a₂N₂",
    line=dict(width=UH_LINE_WIDTH, color="#1f77b4", dash="solid")
)

# Optional: annotate node indices under the axis
if SHOW_NODE_LABELS:
    span = YRANGE[1] - YRANGE[0]
    y_annot = YRANGE[0] - 0.05 * span
    for j, xj in enumerate(nodes, start=1):  # 1-based
        fig.add_annotation(
            x=float(xj), y=y_annot, xref="x", yref="y",
            text=f"node {j}", showarrow=False,
            font=dict(size=11, color="#555")
        )

# Axes limits
fig.update_yaxes(range=list(YRANGE))
fig.update_xaxes(range=[0, 1])

# ---------------- DOF sliders (only 2) ----------------
sliders = [
    widgets.FloatSlider(
        value=float(a0[i]), min=-2.0, max=2.0, step=0.2,
        description=f"a{i+1}", continuous_update=True, readout_format=".2f",
        layout=widgets.Layout(width='320px')
    )
    for i in range(nn)
]
controls = widgets.VBox(sliders)

def update_all(_=None):
    a = np.array([s.value for s in sliders], dtype=float)
    uh = assemble_u(a)
    with fig.batch_update():
        fig.data[idx_uh].y = uh  # hats remain unchanged; only u_h updates

# Wire callbacks
for s in sliders:
    s.observe(update_all, names="value")

# Initial render
update_all()
display(fig, controls)

FigureWidget({
    'data': [{'line': {'color': '#d62728', 'dash': 'solid', 'width': 1.5},
              'mode': 'lines',
              'name': 'N_1(x)',
              'type': 'scatter',
              'uid': 'ec269b2f-6c1a-414f-831d-9eac0dcf7cac',
              'x': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAAAA8D99/62/zPfvP/r+W3+Z7+' ... 'GkgGZgPwAGAaSAZlA/AAAAAAAAAAA='),
                    'dtype': 'f8'}},
             {'line': {'color': '#2ca02c', 'dash': 'solid', 'width': 1.5},
              'mode': 'lines',
              'name': 'N_2(x)',
              'type': 'scatter',
              'uid': '0daa9b88-e055-41b5-81ff-d3e177babc0d',
              'x': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
    

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 5)

This plot illustrates the same as the previous one but now using four elements instead.

In [2]:
# --- P1 hats (unscaled, dashed) + piecewise approximation u_h (solid) ---
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from plotly.io import renderers

# If LaTeX doesn't render in VS Code, try: renderers.default = "iframe_connected"
renderers.default = "notebook_connected"

# ---------------- User settings ----------------
HAT_LINE_WIDTH = 1.5      # thickness for each (unscaled) hat N_i (dashed)
UH_LINE_WIDTH  = 4.0      # thickness for u_h(x) (solid)
PALETTE = ["#d62728", "#2ca02c", "#9467bd", "#8c564b", "#e377c2", "#17becf", "#bcbd22"]
SHOW_NODE_LABELS = True   # annotate nodes 1..nn under the axis
YRANGE = (-2.1, 2.1)      # adjust if you widen DOF slider range

# ---------------- Mesh ----------------
nodes = np.linspace(0.0, 1.0, 5)   # 4 elements -> 5 nodes
nn = len(nodes)
x = np.linspace(0.0, 1.0, 1000)    # dense grid

# ---------------- Global P1 (hat) basis ----------------
def hat_basis(i, x, nodes):
    """
    Global P1 hat function at node i (support on [x_{i-1}, x_{i+1}]).
    N_i(x_i) = 1; linear to 0 at neighboring nodes; zero elsewhere.
    """
    N = np.zeros_like(x, dtype=float)
    xi = nodes[i]
    if i > 0:
        m = (x >= nodes[i-1]) & (x <= xi)
        N[m] = (x[m] - nodes[i-1]) / (xi - nodes[i-1])
    if i < len(nodes) - 1:
        m = (x >= xi) & (x <= nodes[i+1])
        N[m] = (nodes[i+1] - x[m]) / (nodes[i+1] - xi)
    return N

# Precompute all hats
N_global = np.vstack([hat_basis(i, x, nodes) for i in range(nn)])  # (nn, len(x))

def assemble_u(a):
    """u_h(x) = sum_i a_i * N_i(x) on the dense grid."""
    return a @ N_global

# ---------------- Initial DOFs ----------------
a0 = np.array([0.5, 1.8, 1.4, 0.4, 0.2])

# ---------------- Figure ----------------
fig = go.FigureWidget(layout=dict(
    title="Shape functions and approximation uₕ(x)",
    xaxis_title="x",
    yaxis_title="value",
    template="plotly_white",
    height=520
))

# 1) Unscaled hats N_i(x), dashed
hat_trace_indices = []
for i in range(nn):
    tr_idx = len(fig.data)
    fig.add_scatter(
        x=x, y=N_global[i],
        mode="lines",
        name=f"N_{i+1}(x)",  # 1-based label
        line=dict(width=HAT_LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid")
    )
    hat_trace_indices.append(tr_idx)

# 2) u_h(x) = Σ a_i N_i(x), solid
idx_uh = len(fig.data)
fig.add_scatter(
    x=x, y=assemble_u(a0),
    mode="lines",
    name="u_h(x) = Σ a_i N_i(x)",
    line=dict(width=UH_LINE_WIDTH, color="#1f77b4", dash="solid")
)

# Optional: annotate node indices under the axis
if SHOW_NODE_LABELS:
    span = YRANGE[1] - YRANGE[0]
    y_annot = YRANGE[0] - 0.05 * span
    for j, xj in enumerate(nodes, start=1):  # 1-based
        fig.add_annotation(
            x=float(xj), y=y_annot, xref="x", yref="y",
            text=f"node {j}", showarrow=False,
            font=dict(size=11, color="#555")
        )

# Axes limits
fig.update_yaxes(range=list(YRANGE))

# ---------------- DOF sliders ----------------
sliders = [
    widgets.FloatSlider(
        value=float(a0[i]), min=-1.0, max=2.0, step=0.2,
        description=f"a{i+1}", continuous_update=True, readout_format=".2f",
        layout=widgets.Layout(width='320px')
    )
    for i in range(nn)
]
controls = widgets.VBox(sliders)

def update_all(_=None):
    a = np.array([s.value for s in sliders], dtype=float)
    uh = assemble_u(a)
    with fig.batch_update():
        fig.data[idx_uh].y = uh  # hats remain unchanged; only u_h updates

# Wire callbacks
for s in sliders:
    s.observe(update_all, names="value")

# Initial render
update_all()
display(fig, controls)

FigureWidget({
    'data': [{'line': {'color': '#d62728', 'dash': 'solid', 'width': 1.5},
              'mode': 'lines',
              'name': 'N_1(x)',
              'type': 'scatter',
              'uid': 'f820b217-87e6-4c1f-8baf-7e14caf51862',
              'x': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAAAA8D/z/bf+Mt/vP+b7b/1lvu' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'f8'}},
             {'line': {'color': '#2ca02c', 'dash': 'solid', 'width': 1.5},
              'mode': 'lines',
              'name': 'N_2(x)',
              'type': 'scatter',
              'uid': 'd161664e-b818-45f9-9f32-8940dd83f67c',
              'x': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAAAAAABoBgGkgGZwP2gGAaSAZo' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
    

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 5)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 5)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 5)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 5)

## Global vs. local shape functions
This plot shows all global shape functions to the left and what the corresponing local/element shape functions look like to the right.

Click on the legend to hide some of the shape functions to see how each shape function is defined over the entire domain.

In [3]:
# --- P1 Global & Local Shape Functions — Solid Hats + Reliable Toggleable Highlight (1-based labels) ---
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from plotly.io import renderers

# If LaTeX doesn't render in VS Code, try: renderers.default = "iframe_connected"
renderers.default = "notebook_connected"

# ---------------- User settings ----------------
LINE_WIDTH = 3.0                 # <--- change to adjust thickness of ALL hat curves
PALETTE = ["#d62728", "#2ca02c", "#9467bd", "#8c564b", "#e377c2", "#17becf", "#bcbd22"]
SHOW_NODE_LABELS = True
SHADE_COLOR_ON  = "rgba(0,0,0,0.12)"  # when highlight is ON
SHADE_LINE_ON   = "rgba(0,0,0,0.25)"
SHADE_COLOR_OFF = "rgba(0,0,0,0)"     # fully transparent when OFF
SHADE_LINE_OFF  = "rgba(0,0,0,0)"

# ---------------- Mesh ----------------
num_nodes = 15
nodes = np.linspace(0.0, 1.0, num_nodes)   # 4 elements -> 5 nodes
nn = len(nodes)                    # number of nodes
ne = nn - 1                        # number of elements
x = np.linspace(0.0, 1.0, 1000)    # dense grid for plotting

# ---------------- Global P1 (hat) basis ----------------
def hat_basis(i, x, nodes):
    """
    Global P1 hat function at node i (support on [x_{i-1}, x_{i+1}]).
    N_i(x_i) = 1; N_i is linear to 0 at neighbors; zero elsewhere.
    """
    N = np.zeros_like(x, dtype=float)
    xi = nodes[i]
    if i > 0:
        m = (x >= nodes[i-1]) & (x <= xi)
        N[m] = (x[m] - nodes[i-1]) / (xi - nodes[i-1])
    if i < len(nodes) - 1:
        m = (x >= xi) & (x <= nodes[i+1])
        N[m] = (nodes[i+1] - x[m]) / (nodes[i+1] - xi)
    return N

# Precompute all global hats on the dense grid
N_global = np.vstack([hat_basis(i, x, nodes) for i in range(nn)])  # (nn, len(x))

# ---------------- Restrict hats to a selected element ----------------
def hats_on_element(e_idx, n_plot=200):
    """
    For element with zero-based index e_idx (between nodes e_idx and e_idx+1),
    return x_e (physical sampling), N_e(x_e), N_{e+1}(x_e).
    """
    x0, x1 = nodes[e_idx], nodes[e_idx + 1]
    xs = np.linspace(x0, x1, n_plot)
    Ne   = hat_basis(e_idx,     xs, nodes)
    Nep1 = hat_basis(e_idx + 1, xs, nodes)
    return xs, Ne, Nep1

def make_rect_shape(x0, x1, fillcolor, linecolor):
    """Build a shaded rectangle shape spanning [x0, x1] in the LEFT subplot."""
    return dict(
        type="rect",
        xref="x1", yref="paper",
        x0=float(x0), x1=float(x1), y0=0.0, y1=1.0,
        fillcolor=fillcolor, line=dict(color=linecolor, width=1)
    )

# ---------------- Figure with two subplots ----------------
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "Global shape functions",
        "Local shape functions"
    ),
    horizontal_spacing=0.12
)
fig = go.FigureWidget(fig)

# ----- Left: all global hat functions N_i(x) as solid lines, named N_1..N_nn -----
global_hat_indices = []
for i in range(nn):
    name_1based = f"N_{i+1}(x)"
    idx = len(fig.data)
    fig.add_scatter(
        x=x, y=N_global[i],
        mode="lines",
        name=name_1based,
        line=dict(width=LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid"),
        row=1, col=1
    )
    global_hat_indices.append(idx)

# Optional: node index labels (1..nn) under the left subplot
if SHOW_NODE_LABELS:
    for j, xj in enumerate(nodes, start=1):  # 1-based label
        fig.add_annotation(
            x=float(xj), y=-0.08, xref="x1", yref="paper",
            text=f"node {j}", showarrow=False,
            font=dict(size=11, color="#555")
        )

# ----- Right: show only N_e and N_{e+1} ON the selected element (solid, matching colors) -----
e0_idx = 0  # zero-based initial element index
xs, Ne, Nep1 = hats_on_element(e0_idx)
color_e   = PALETTE[e0_idx % len(PALETTE)]
color_ep1 = PALETTE[(e0_idx + 1) % len(PALETTE)]

idx_right_Ne = len(fig.data)
fig.add_scatter(
    x=xs, y=Ne,
    mode="lines", name=f"N_{e0_idx+1}(x) on element {e0_idx+1}",
    line=dict(color=color_e, width=LINE_WIDTH, dash="solid"),
    row=1, col=2
)
idx_right_Nep1 = len(fig.data)
fig.add_scatter(
    x=xs, y=Nep1,
    mode="lines", name=f"N_{e0_idx+2}(x) on element {e0_idx+1}",
    line=dict(color=color_ep1, width=LINE_WIDTH, dash="solid"),
    row=1, col=2
)

# ----- Add ONE rectangle shape once; we'll toggle by changing its colors -----
x0, x1 = nodes[e0_idx], nodes[e0_idx + 1]
fig.update_layout(shapes=[make_rect_shape(x0, x1, SHADE_COLOR_ON, SHADE_LINE_ON)])

# ----- Styling -----
fig.update_layout(
    template="plotly_white",
    height=560,
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="left", x=0),
    title="Global and local shape functions"
)
fig.update_xaxes(title_text="x", row=1, col=1)
fig.update_yaxes(title_text="shape value", row=1, col=1, range=[-0.05, 1.05])
fig.update_xaxes(title_text="x", row=1, col=2)
fig.update_yaxes(title_text="shape value", row=1, col=2, range=[-0.05, 1.05])

# ---------------- Controls ----------------
elem_slider = widgets.IntSlider(
    value=e0_idx + 1, min=1, max=ne, step=1,
    description="Element", continuous_update=True,
    layout=widgets.Layout(width='280px')
)
toggle_highlight = widgets.Checkbox(
    value=True, description="Highlight element region (left)", indent=False
)
elem_info = widgets.HTML()

def update_view(_=None):
    e1_based = int(elem_slider.value)  # 1..ne
    e_idx = e1_based - 1               # 0..ne-1

    # Right subplot: N_e and N_{e+1} restricted to element e (colors match global)
    xs, Ne, Nep1 = hats_on_element(e_idx)
    color_e   = PALETTE[e_idx % len(PALETTE)]
    color_ep1 = PALETTE[(e_idx + 1) % len(PALETTE)]

    # Update left shaded region (we keep shape[0] always; just move/toggle its colors)
    x0, x1 = nodes[e_idx], nodes[e_idx + 1]
    with fig.batch_update():
        # Move the rectangle
        fig.layout.shapes[0].x0 = float(x0)
        fig.layout.shapes[0].x1 = float(x1)
        # Toggle via colors (reliable in VS Code)
        if bool(toggle_highlight.value):
            fig.layout.shapes[0].fillcolor = SHADE_COLOR_ON
            fig.layout.shapes[0].line.color = SHADE_LINE_ON
        else:
            fig.layout.shapes[0].fillcolor = SHADE_COLOR_OFF
            fig.layout.shapes[0].line.color = SHADE_LINE_OFF

        # Update right subplot curves and names
        fig.data[idx_right_Ne].x = xs
        fig.data[idx_right_Ne].y = Ne
        fig.data[idx_right_Ne].line.color = color_e
        fig.data[idx_right_Ne].name = f"N_{e_idx+1}(x) on element {e_idx+1}"

        fig.data[idx_right_Nep1].x = xs
        fig.data[idx_right_Nep1].y = Nep1
        fig.data[idx_right_Nep1].line.color = color_ep1
        fig.data[idx_right_Nep1].name = f"N_{e_idx+2}(x) on element {e_idx+1}"

    # Info text
    nL = e_idx + 1
    nR = e_idx + 2
    elem_info.value = (
        f"<b>Selected element:</b> {e1_based} &nbsp; | &nbsp; "
        f"<b>nodes:</b> {nL}–{nR} &nbsp; | &nbsp; "
        f"<b>x-interval:</b> [{nodes[e_idx]:.3f}, {nodes[e_idx+1]:.3f}]"
    )

# Wire callbacks
elem_slider.observe(update_view, names="value")
toggle_highlight.observe(update_view, names="value")

# Initial draw
update_view()
display(fig, widgets.HBox([elem_slider, toggle_highlight, elem_info]))

FigureWidget({
    'data': [{'line': {'color': '#d62728', 'dash': 'solid', 'width': 3.0},
              'mode': 'lines',
              'name': 'N_1(x)',
              'type': 'scatter',
              'uid': '16710abb-4f31-4c9f-9e87-6f30f5197471',
              'x': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAA8D/S+IN7Mo3vP6fxB/dkGu' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'line': {'color': '#2ca02c', 'dash': 'solid', 'width': 3.0},
              'mode': 'lines',
              'name': 'N_2(x)',
              'type': 'scatter',
              'uid': '9f840f6f-65ae-4ee3-8cdd-cdb287ac5914',
              'x': {'bdata': ('AAAAAAAAAABoBgGkgGZQP2gGAaSAZm' ... 't/me/vP33/rb/M9+8/AAAAAAAA8D8='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'b